## Introduction

In labor economics an important question is what determines the wage of workers. This is a causal question,
but we could begin to investigate from a predictive perspective.

In the following wage example, $Y$ is the hourly wage of a worker and $X$ is a vector of worker's characteristics, e.g., education, experience, gender. Two main questions here are:


* How to use job-relevant characteristics, such as education and experience, to best predict wages?

* What is the difference in predicted wages between men and women with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

## Data


The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015.  We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors;  individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below $3$. 

The variable of interest $Y$ is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single workers (never married),some high school(shs) and high school graduate(hsg). The final sample size is $n= $1376.

## EXPLAINING THE IDEA OF SAMPLE SPLITTING

<p style='text-align: justify;'> When we perform a regression, the estimated coefficients are adjusted so that they are reduced to the minimum mean squared error, i.e., these coefficients perform well within the sample. However, if we want to know the predictive power of the estimated coefficients, it is necessary to measure their out-of-sample performance. To do this, we use the idea of sample splitting, which tells us that we have to divide our sample into two groups: training and test. This division is random. However, the proportion of each group to the total is chosen by the researcher. First, the training sample is used to estimate the coefficients of our model to know the prediction rule. Then, the test sample is used to evaluate the quality of the prediction rule, i.e., we find the predicted values of the endogenous variable using the coefficients obtained from the training sample. Finally, we have to calculate the out-of-sample mean square error ($MSE_{test}$) and $R^{2}_{test}$ in the test sample.

For example, suppose we have the following wage model:  $ lwage \sim sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2$ and data size is $n$. First, we have to divide the data into two groups randomly. This division can be $4/5n$ as the training sample, $M_{train}$, and  $1/5n$ as the test sample, $M_{test}$,. After that, we regress the model on $M_{train}$ and obtain the estimated coefficients $\hat\beta$. With the estimated coefficients, we predict $lwage$ in $M_{test}$, and we calculate the out-of-sample mean square error $MSE_{test}$ and $R^{2}_{test}$ in $M_{test}$.After that, we can analyze whether the $MSE_{test}$ is quite closed to the $MSE_{sample}$ (obtained from regressing the model on the complete sample) or the discrepancy is quite large.

## Data analysis

We start by loading the data set.

In [1]:
library(dplyr)
library(hdm)

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'hdm' was built under R version 3.6.3"

In [2]:
load("../data/wage2015_subsample_inference.Rdata")
dim(data)

[1] 5150   20

In [3]:
# filtering the data for values where shs=1 or hsg=1(workers with some high school(shs) or high school graduate(hsg) )
data=filter(data,shs==1|hsg==1)
dim(data)

[1] 1376   20

Let's have a look at the structure of the data.

We are constructing the output variable $Y$ and the matrix $Z$ which includes the characteristics of workers that are given in the data.

In [4]:
Y <- log(data$wage)
n <- length(Y)
Z <- data[-which(colnames(data) %in% c("wage","lwage"))]
p <- dim(Z)[2]

cat("Number of observation:", n, '\n')
cat( "Number of raw regressors:", p)

Number of observation: 1376 
Number of raw regressors: 18

For the outcome variable *wage* and a subset of the raw regressors, we calculate the empirical mean to get familiar with the data.

In [5]:
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")

In [6]:
library(xtable)

Z_subset <- data[which(colnames(data) %in% c("lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"))]
table <- matrix(0, 12, 1)
table[1:12,1]   <- as.numeric(lapply(Z_subset,mean))
rownames(table) <- c("Log Wage","Sex","Some High School","High School Graduate","Some College","College Graduate", "Advanced Degree","Midwest","South","West","Northeast","Experience")
colnames(table) <- c("Sample mean")
tab<- xtable(table, digits = 2)
tab

Sample mean
Log Wage              2.7185624 
Sex                   0.3219477 
Some High School      0.0872093 
High School Graduate  0.9127907 
Some College          0.0000000 
College Graduate      0.0000000 
Advanced Degree       0.0000000 
Midwest               0.2863372 
South                 0.2914244 
West                  0.1984012 
Northeast             0.2238372 
Experience           17.1900436

E.g., the share of female workers in our sample is ~32% ($sex=1$ if female).

Alternatively, we can also print the table as latex.

## Prediction Question

Now, we will construct a prediction rule for hourly wage $Y$, which depends linearly on job-relevant characteristics $X$:

\begin{equation}\label{decompose}
Y = \beta'X+ \epsilon.
\end{equation}

Our goals are

* Predict wages  using various characteristics of workers.

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.


We employ two different specifications for prediction:


1. Basic Model:   $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators,  occupation and industry indicators, regional indicators).


2. Flexible Model:  $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g., ${exp}^2$ and ${exp}^3$) and additional two-way interactions of polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is *experience* times the indicator of having a *college degree*.

Using the **Flexible Model**, enables us to approximate the real relationship by a
 more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.
 
 However, this model incorporates a larger number of regressors and requires a larger number of observations to have sufficient degrees of freedom.  
 

Now, let us fit both models to our data by running ordinary least squares (ols):

In [7]:
# 1. basic model
basic <- lwage~ (sex+exp1 + hsg+ mw + so + we +occ2+ind2)
regbasic <- lm(basic, data=data)
regbasic # estimated coefficients
cat( "Number of regressors in the basic model:",length(regbasic$coef), '\n') # number of regressors in the Basic Model



Call:
lm(formula = basic, data = data)

Coefficients:
(Intercept)          sex         exp1          hsg           mw           so  
  2.7518725   -0.0733094    0.0075742    0.0811342   -0.0431882   -0.1091620  
         we        occ22        occ23        occ24        occ25        occ26  
  0.0129620   -0.1961261   -0.0086113    0.0005078    0.2615289   -0.3510072  
      occ27        occ28        occ29       occ210       occ211       occ212  
 -0.1900342   -0.6616521   -0.3013316   -0.0576220   -0.4176903   -0.4663571  
     occ213       occ214       occ215       occ216       occ217       occ218  
 -0.4219896   -0.5527766   -0.4747648   -0.2381724   -0.3529422   -0.3976108  
     occ219       occ220       occ221       occ222        ind23        ind24  
 -0.1181885   -0.1053967   -0.1737437   -0.3479965    0.1742747    0.0504201  
      ind25        ind26        ind27        ind28        ind29       ind210  
  0.0585330    0.0348081    0.2379530    0.0922050    0.0928608    0.2399036

Number of regressors in the basic model: 48 


##### Note that the basic model consists of $48$ regressors.

In [8]:
# 2. flexible model
flex <- lwage ~ (exp1+exp2+exp3+exp4+hsg+occ2+ind2+mw+so+we)**2
regflex <- lm(flex, data=data)
regflex # estimated coefficients
cat( "Number of regressors in the flexible model:",length(regflex$coef)) # number of regressors in the Flexible Model


Call:
lm(formula = flex, data = data)

Coefficients:
  (Intercept)           exp1           exp2           exp3           exp4  
    1.544e+01     -3.684e+00      4.183e+01     -2.251e+01      7.480e+00  
          hsg          occ22          occ23          occ24          occ25  
    5.723e-01     -2.441e+00     -4.590e+01      3.814e+00     -5.081e+01  
        occ26          occ27          occ28          occ29         occ210  
    1.803e+01     -9.997e-01     -1.024e+01     -2.064e+01     -3.857e+00  
       occ211         occ212         occ213         occ214         occ215  
    7.938e-01      2.424e+00     -1.720e+00     -6.556e-01     -1.048e+01  
       occ216         occ217         occ218         occ219         occ220  
    6.067e-01     -4.943e+00     -3.445e-01     -6.796e+00     -3.114e+00  
       occ221         occ222          ind23          ind24          ind25  
   -1.176e+00     -1.987e+00     -1.653e+01     -5.930e+00     -1.193e+01  
        ind26          ind27      

Number of regressors in the flexible model: 826

Note that the flexible model consists of $826$ regressors for only 1376 obs.

LASSO

In [9]:
#install.packages('hdm')
#lambdaCalculation(penalty = list(homoscedastic = FALSE, X.dependent.lambda =
#FALSE, lambda.start = NULL, c = 1.1, gamma = 0.1), y = data$wage, x =data)

In [10]:
#library(hdm)
flex <- lwage ~ (exp1+exp2+exp3+exp4+hsg+occ2+ind2+mw+so+we)**2
lassoreg<- rlasso(flex, data=data,post=FALSE)

sumlasso<- summary(lassoreg)



Call:
rlasso.formula(formula = flex, data = data, post = FALSE)

Post-Lasso Estimation:  FALSE 

Total number of variables: 825
Number of selected variables: 24 

Residuals: 
     Min       1Q   Median       3Q      Max 
-1.37420 -0.29937 -0.01777  0.28156  3.54523 

              Estimate
(Intercept)      2.680
exp1             0.003
exp2             0.000
exp3             0.000
exp4             0.000
hsg              0.000
occ22            0.000
occ23            0.000
occ24            0.000
occ25            0.000
occ26            0.000
occ27            0.000
occ28           -0.085
occ29            0.000
occ210           0.000
occ211           0.000
occ212           0.000
occ213          -0.173
occ214          -0.142
occ215          -0.153
occ216           0.000
occ217          -0.030
occ218           0.000
occ219           0.000
occ220           0.000
occ221           0.000
occ222           0.000
ind23            0.000
ind24            0.000
ind25            0.000
ind26            0

Flex lasso model with 24 regressors

Now, we can evaluate the performance of the three models based on the (adjusted) $R^2_{sample}$ and the (adjusted) $MSE_{sample}$.The aim is to find an $R^2$ close to 1 and low $MSE$ .

In [11]:
# Assess the predictive performance

sumbasic <- summary(regbasic)
sumflex <- summary(regflex)

#  R-squared-basic model
R2.1 <- sumbasic$r.squared
cat("R-squared for the basic model: ", R2.1, "\n")
R2.adj1 <- sumbasic$adj.r.squared
cat("adjusted R-squared for the basic model: ", R2.adj1, "\n")

#  R-squared-flex model
R2.2 <- sumflex$r.squared
cat("R-squared for the flexible model: ", R2.2, "\n")
R2.adj2 <- sumflex$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adj2, "\n")

#  R-squared-flex lasso model
R2.L <- sumlasso$r.squared
cat("R-squared for the lasso with flexible model: ", R2.L, "\n")
R2.adjL <- sumlasso$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adjL, "\n")

R-squared for the basic model:  0.1802381 
adjusted R-squared for the basic model:  0.1512255 
R-squared for the flexible model:  0.507044 
adjusted R-squared for the flexible model:  0.2315028 
R-squared for the lasso with flexible model:  0.1157304 
adjusted R-squared for the flexible model:  0.1000216 


In [12]:
# calculating the MSE-basic model
MSE1 <- mean(sumbasic$res^2)
cat("MSE for the basic model: ", MSE1, "\n")

p1 <- sumbasic$df[1] # number of regressors
MSE.adj1 <- (n/(n-p1))*MSE1
cat("adjusted MSE for the basic model: ", MSE.adj1, "\n")

# calculating the MSE-flex model
MSE2 <-mean(sumflex$res^2)
cat("MSE for the flexible model: ", MSE2, "\n")
p2 <- sumflex$df[1]
MSE.adj2 <- (n/(n-p2))*MSE2
cat("adjusted MSE for the flexible model: ", MSE.adj2, "\n")

# calculating the MSE-flex lasso model
MSEL <-mean(sumlasso$res^2)
cat("MSE for the lasso flexible model: ", MSEL, "\n")
pL <- length(sumlasso$coef)
MSE.adjL <- (n/(n-pL))*MSEL
cat("adjusted MSE for the lasso flexible model: ", MSE.adjL, "\n")

MSE for the basic model:  0.2082191 
adjusted MSE for the basic model:  0.2157451 
MSE for the flexible model:  0.1252106 
adjusted MSE for the flexible model:  0.1953398 
MSE for the lasso flexible model:  0.224604 
adjusted MSE for the lasso flexible model:  0.5619184 


In [13]:
library(xtable)
table <- matrix(0, 3, 5)
table[1,1:5]   <- c(p1,R2.1,MSE1,R2.adj1,MSE.adj1)
table[2,1:5]   <- c(p2,R2.2,MSE2,R2.adj2,MSE.adj2)
table[3,1:5]   <- c(pL,R2.L,MSEL,R2.adjL,MSE.adjL)
colnames(table)<- c("p","$R^2_{sample}$","$MSE_{sample}$","$R^2_{adjusted}$", "$MSE_{adjusted}$")
rownames(table)<- c("basic reg","flexible reg", "lasso flex")
tab<- xtable(table, digits =c(0,0,2,2,2,2))
print(tab,type="latex") # type="latex" for printing table in LaTeX
tab

% latex table generated in R 3.6.1 by xtable 1.8-4 package
% 
\begin{tabular}{rrrrrr}
  \hline
 & p & \$R\verb|^|2\_\{sample\}\$ & \$MSE\_\{sample\}\$ & \$R\verb|^|2\_\{adjusted\}\$ & \$MSE\_\{adjusted\}\$ \\ 
  \hline
basic reg & 48 & 0.18 & 0.21 & 0.15 & 0.22 \\ 
  flexible reg & 494 & 0.51 & 0.13 & 0.23 & 0.20 \\ 
  lasso flex & 826 & 0.12 & 0.22 & 0.10 & 0.56 \\ 
   \hline
\end{tabular}


p   $R^2_{sample}$ $MSE_{sample}$ $R^2_{adjusted}$
basic reg     48 0.1802381      0.2082191      0.1512255       
flexible reg 494 0.5070440      0.1252106      0.2315028       
lasso flex   826 0.1157304      0.2246040      0.1000216       
             $MSE_{adjusted}$
basic reg    0.2157451       
flexible reg 0.1953398       
lasso flex   0.5619184

Considering all measures above, the flex model performs  better than the other models.

One procedure to circumvent this issue is to use **data splitting** that is described and applied in the following.

## Data Splitting

Measure the prediction quality of the two models via data splitting:

- Randomly split the data into one training sample and one testing sample. Here we just use a simple method (stratified splitting is a more sophiscticated version of splitting that we can consider).
- Use the training sample for estimating the parameters of the Basic Model and the Flexible Model.
- Use the testing sample for evaluation. Predict the $\mathtt{wage}$  of every observation in the testing sample based on the estimated parameters in the training sample.
- Calculate the Mean Squared Prediction Error $MSE_{test}$ based on the testing sample for both prediction models. 

In [14]:
#splitting the data

set.seed(1) # to make the results replicable (generating random numbers)
random_2 <- sample(1:n, floor(n*4/5))

# draw (4/5)*n random numbers from 1 to n without replacing them
train <- data[random_2,] # training sample
test <- data[-random_2,] # testing sample
dim(train)
dim(test)
#1100 obs for train and 276 for test

[1] 1100   20

[1] 276  20

In [15]:
# basic model
# estimating the parameters in the training sample
regbasic <- lm(basic, data=train)
regbasic


Call:
lm(formula = basic, data = train)

Coefficients:
(Intercept)          sex         exp1          hsg           mw           so  
   2.790397    -0.113776     0.008171     0.045630    -0.007887    -0.092965  
         we        occ22        occ23        occ24        occ25        occ26  
   0.027930    -0.198916    -0.014664     0.202306     0.242367     0.006158  
      occ27        occ28        occ29       occ210       occ211       occ212  
  -0.180164    -0.796769    -0.241348    -0.015354    -0.399688    -0.506190  
     occ213       occ214       occ215       occ216       occ217       occ218  
  -0.446041    -0.575386    -0.450461    -0.209308    -0.338949    -0.251586  
     occ219       occ220       occ221       occ222        ind23        ind24  
  -0.137541    -0.124913    -0.158960    -0.350055     0.191507     0.036101  
      ind25        ind26        ind27        ind28        ind29       ind210  
   0.034002    -0.004881     0.214846     0.042589     0.076761     0.25978

In [16]:
# calculating the out-of-sample MSE
trainregbasic <- predict(regbasic, newdata=test)
trainregbasic

43       47      158      248      261      482      858      936 
2.872798 2.475568 2.909436 2.843801 2.706183 2.816292 3.030303 2.580716 
     954     1323     1490     1567     1630     1870     1964     2025 
2.801527 2.787998 3.062171 2.836898 2.326333 2.979703 2.285942 2.276830 
    2063     2362     2674     2925     3021     3065     3084     3170 
2.412655 2.711248 2.784423 2.636268 2.744333 2.677121 2.840804 3.074304 
    3385     3395     3581     3639     3715     3759     3770     3797 
2.828920 3.123457 2.388790 2.799951 3.210152 2.792775 2.264362 2.738699 
    3876     3917     4182     4183     4198     4222     4417     4610 
2.615367 2.817868 2.850550 2.798970 2.905476 2.129962 2.997718 2.490984 
    4671     4809     4869     4940     5069     5073     5180     5224 
2.297529 2.774117 2.733286 2.812450 2.890822 2.827126 2.760961 2.889827 
    5344     5598     5801     5848     6122     6236     6314     6533 
2.848974 3.119391 2.354723 2.940427 2.762220 2.724100 2.860109 2.901859 
    6844     6874     6881     7034     7137     7171     7309     7322 
2.265131 2.460612 2.263418 3.180935 2.501392 2.390470 2.795458 2.496359 
    7335     7381     7389     7929     7998     8136     8350     8471 
2.809982 2.997905 2.803136 2.738447 2.526222 2.960109 2.769129 2.708267 
    8478     8517     8572     8574     8578     8647     8802     8880 
2.664675 2.806314 2.703490 2.958472 2.820964 2.734791 2.906452 2.785470 
    9002     9111     9154     9360     9432     9456     9469     9488 
2.464904 2.993126 2.318446 2.779223 2.921167 2.636552 2.930964 2.704377 
    9504     9731     9770     9771     9852     9980    10208    10445 
2.421110 2.575738 2.505990 2.752206 2.809982 2.695625 2.825308 2.378906 
   10464    10709    10797    10905    11286    11490    11526    11542 
2.793640 2.795029 2.962288 2.826964 2.841088 2.933928 2.747153 2.839341 
   11858    12271    12361    12418    12439    12731    12753    12764 
3.083966 2.808405 2.480362 2.686605 2.844052 2.793640 2.986799 2.733305 
   12786    12867    12895    12980    13040    13105    13221    13354 
2.569409 2.760837 2.705152 2.948344 2.730090 2.968491 2.502339 2.901246 
   13358    13421    13603    13616    13728    13767    13858    13859 
2.482359 3.125308 2.160298 2.746005 2.666227 2.802424 2.752024 3.040440 
   14106    14134    14141    14189    14252    14326    14337    14683 
2.722042 2.407710 2.389074 2.669847 2.595581 2.843988 2.675921 2.824746 
   14692    14705    14748    14919    14943    14956    15015    15071 
2.876767 2.885334 2.712918 2.734495 2.450420 2.782129 2.572474 2.693666 
   15105    15193    15253    15295    15299    15385    15508    16328 
2.440899 2.458114 3.381992 2.756010 2.720160 3.187121 2.502988 2.643000 
   16365    16523    16841    17079    17152    17177    17249    17402 
2.535670 2.380429 2.893551 2.893823 2.723327 2.757586 2.886939 2.268831 
   18230    18257    18336    18375    18528    18558    18700    18846 
2.326511 2.729321 2.714885 2.658776 2.715157 2.387997 2.578446 2.620074 
   19056    19152    19181    19263    19615    19848    19987    19995 
2.845393 2.988053 2.934211 2.734891 2.547116 2.409090 2.303548 2.521727 
   20049    20062    20116    20198    20410    20423    20456    20653 
2.788741 2.619299 2.747839 2.702963 2.651368 2.899300 2.253588 2.704025 
   20777    20916    21030    21543    21626    21929    22080    22721 
2.479894 2.623468 2.858322 3.296119 2.892065 2.588540 2.680694 2.521727 
   22734    23053    23146    23353    23405    23517    23848    23991 
2.690374 2.718013 2.293828 2.776616 2.815392 2.704888 2.670543 2.800256 
   24374    24514    24649    25233    25417    25519    25767    25804 
3.132884 2.424292 2.569726 2.293828 1.983952 2.414019 2.866345 2.841260 
   25937    26067    26135    26331    26426    26620    26623    26696 
3.049481 2.589698 2.917757 3.106391 2.950831 2.512371 2.744083 2.750562 
   26804    26936    27200    27379    27395    27491    27541  

In [17]:
y.test <- log(test$wage)
MSE.test1 <- sum((y.test-trainregbasic)^2)/length(y.test)
R2.test1<- 1- MSE.test1/var(y.test)
R2.test1_adj<-1- MSE.test1/var(y.test)*(length(y.test)/(length(y.test)-51))
MSE.test1_adj<-sum((y.test-trainregbasic)^2)*(length(y.test)/(length(y.test)-51))
cat("Test MSE for the basic model: ", MSE.test1, " ")
cat("Test MSE_adj for the basic model: ", MSE.test1_adj, " ")
cat("Test R2 for the basic model: ", R2.test1, " ")

cat("Test R2_adj for the basic model: ", R2.test1_adj)

Test MSE for the basic model:  0.2015247  Test MSE_adj for the basic model:  68.22819  Test R2 for the basic model:  0.06784807  Test R2_adj for the basic model:  -0.1434397

In the basic model, the $MSE_{test}$ is quite closed to the $MSE_{sample}$ (0.2082191). however, the $MSEadj_{test}$ is very different from the $MSEadj_{sample}$  which indicates that there is a strong penalty for the number of regressors.

In [18]:
# flexible model
# estimating the parameters

regflex <- lm(flex, data=train)

# calculating the out-of-sample MSE
trainregflex<- predict(regflex, newdata=test)
y.test <- log(test$wage)
MSE.test2 <- sum((y.test-trainregflex)^2)/length(y.test)
R2.test2<- 1- MSE.test2/var(y.test)

cat("Test MSE for the flexible model: ", MSE.test2, " ")

cat("Test R2 for the flexible model: ", R2.test2)

Warning message in predict.lm(regflex, newdata = test):
"prediction from a rank-deficient fit may be misleading"

Test MSE for the flexible model:  22576.15  Test R2 for the flexible model:  -104424.9

Moreover, in this model the values of T and F are high and this is because we have a large number of regressors that are close to the number of observations we have.

In the flexible model, the discrepancy between the $MSE_{test}$ and the $MSE_{sample}$ is  large.

It is worth to notice that the $MSE_{test}$ vary across different data splits. Hence, it is a good idea average the out-of-sample MSE over different data splits to get valid results.

Nevertheless, we observe that, based on the out-of-sample $MSE$, the flex lasso model  performs is about as well (or slightly better) than the basic model and flexible model. 


Next, let us use lasso regression in the flexible model instead of ols regression. Lasso (*least absolute shrinkage and selection operator*) is a penalized regression method that can be used to reduce the complexity of a regression model when the number of regressors $p$ is relatively large in relation to $n$. 

Note that the out-of-sample $MSE$ on the test sample can be computed for any other black-box prediction method as well. Thus, let us finally compare the performance of lasso regression in the flexible model to ols regression.

In [19]:
# flexible model using lasso

# estimating the parameters
#library(hdm)
reglasso <- rlasso(flex, data=train, post=FALSE)

# calculating the out-of-sample MSE
trainreglasso<- predict(reglasso, newdata=test)
MSE_flex.lasso <- sum((y.test-trainreglasso)^2)/length(y.test)
R2_flex.lasso<- 1- MSE_flex.lasso/var(y.test)


cat("Test MSE for the lasso on flexible model: ", MSE_flex.lasso, " ")

cat("Test R2 for the lasso flexible model: ", R2_flex.lasso)

Test MSE for the lasso on flexible model:  0.2032003  Test R2 for the lasso flexible model:  0.06009756

In [20]:
summary(reglasso)


Call:
rlasso.formula(formula = flex, data = train, post = FALSE)

Post-Lasso Estimation:  FALSE 

Total number of variables: 825
Number of selected variables: 17 

Residuals: 
     Min       1Q   Median       3Q      Max 
-1.37748 -0.29748 -0.02832  0.27519  3.51903 

              Estimate
(Intercept)      2.681
exp1             0.004
exp2             0.000
exp3             0.000
exp4             0.000
hsg              0.000
occ22            0.000
occ23            0.000
occ24            0.000
occ25            0.000
occ26            0.000
occ27            0.000
occ28           -0.259
occ29            0.000
occ210           0.000
occ211           0.000
occ212           0.000
occ213          -0.191
occ214          -0.141
occ215          -0.162
occ216           0.000
occ217           0.000
occ218           0.000
occ219           0.000
occ220           0.000
occ221           0.000
occ222           0.000
ind23            0.000
ind24            0.000
ind25            0.000
ind26            

In [21]:
# basic model using lasso

# estimating the parameters
#library(hdm)
reglasso_basic <- rlasso(basic, data=train, post=FALSE)

# calculating the out-of-sample MSE
trainreglasso_basic<- predict(reglasso_basic, newdata=test)
MSE_basic.lasso <- sum((y.test-trainreglasso_basic)^2)/length(y.test)
R2_basic.lasso<- 1- MSE_basic.lasso/var(y.test)


cat("Test MSE for the lasso on flexible model: ", MSE_basic.lasso, " ")

cat("Test R2 for the lasso flexible model: ", R2_basic.lasso)

Test MSE for the lasso on flexible model:  0.2051146  Test R2 for the lasso flexible model:  0.05124275

In [22]:
summary(reglasso_basic)


Call:
rlasso.formula(formula = basic, data = train, post = FALSE)

Post-Lasso Estimation:  FALSE 

Total number of variables: 47
Number of selected variables: 14 

Residuals: 
     Min       1Q   Median       3Q      Max 
-1.34220 -0.29244 -0.02897  0.28715  3.47008 

            Estimate
(Intercept)    2.696
sex           -0.072
exp1           0.007
hsg            0.000
mw             0.000
so            -0.043
we             0.000
occ22          0.000
occ23          0.000
occ24          0.000
occ25          0.000
occ26          0.000
occ27          0.000
occ28         -0.315
occ29          0.000
occ210         0.000
occ211        -0.071
occ212        -0.036
occ213        -0.214
occ214        -0.220
occ215        -0.176
occ216         0.000
occ217        -0.037
occ218         0.000
occ219         0.000
occ220         0.000
occ221         0.000
occ222        -0.015
ind23          0.000
ind24          0.000
ind25          0.000
ind26          0.000
ind27          0.000
ind28          0

Finally, let us summarize the results:

In [23]:
table2 <- matrix(0, 4,2)
table2[1,1]   <- MSE.test1
table2[2,1]   <- MSE.test2
table2[3,1]   <- MSE_flex.lasso
table2[1,2]   <- R2.test1
table2[2,2]   <- R2.test2
table2[3,2]   <- R2_flex.lasso
table2[4,1]<-MSE_basic.lasso
table2[4,2]<-R2_basic.lasso
rownames(table2)<- c("basic reg","flexible reg","lasso flex regression","lasso basic regression")
colnames(table2)<- c("$MSE_{test}$", "$R^2_{test}$")
tab2 <- xtable(table2, digits =4)
tab2

$MSE_{test}$ $R^2_{test}$ 
basic reg              2.015247e-01  6.784807e-02
flexible reg           2.257615e+04 -1.044249e+05
lasso flex regression  2.032003e-01  6.009756e-02
lasso basic regression 2.051146e-01  5.124275e-02

It is observed that the best model is the basic flex model, however, both R and MSE are values that we would not want because they indicate that there is not enough information in our data to correctly predict the wage of a worker, therefore, we should increase the number of observations and restrict the number of regressors. 

# Lasso

## Partialling Out using Lasso

## Case 1

We use Lasso when the dimension of $W$ is high, this allows us to regulize the model selecting some variables.

In this case, we will use the following matrix $W$:

Matrix $W$ = 'exp1 + hsg + mw + so + we + occ2+ ind2'

In [24]:
library(sandwich)

In [25]:
flex.y <- lwage ~ exp1+hsg+mw+so+we+occ2+ind2 #model for y

flex.d <- sex ~ exp1+hsg+mw+so+we+occ2+ind2 #model for D

#partialling-out the linear effect of W from Y

t.Y <- rlasso(flex.y, data=data)$res

#partialling-out the linear effect of W from D

t.D <- rlasso(flex.d, data=data)$res

#regression of Y on D after partialling out the effect of W
partial.lasso.fit <- lm(t.Y~t.D)
partial.lasso.est_1 <- summary(partial.lasso.fit)$coef[2,1]

cat('Coefficient for D via partialling-out', partial.lasso.est_1)

#standard error

HCV.coefs <- vcovHC(partial.lasso.fit, type='HC')
partial.se_1 <- sqrt(diag(HCV.coefs))[2]

Coefficient for D via partialling-out -0.09065628

With Lasso the coefficient for $D$ is $-0.09065628$

## Case 2

In this case, we will use the following matrix $W$:

Matrix $W$ =  (exp1+exp2+exp3+exp4+hsg+occ2+ind2+mw+so+we)**2

In [26]:
flex.y <- lwage ~ (exp1+exp2+exp3+exp4+hsg+mw+so+we+occ2+ind2)**2 #model for y

flex.d <- sex ~ (exp1+exp2+exp3+exp4+hsg+mw+so+we+occ2+ind2)**2 #model for D

#partialling-out the linear effect of W from Y

t.Y <- rlasso(flex.y, data=data)$res

#partialling-out the linear effect of W from D

t.D <- rlasso(flex.d, data=data)$res

#regression of Y on D after partialling out the effect of W
partial.lasso.fit <- lm(t.Y~t.D)
partial.lasso.est_2 <- summary(partial.lasso.fit)$coef[2,1]

cat('Coefficient for D via partialling-out', partial.lasso.est_2)


#standard error

HCV.coefs <- vcovHC(partial.lasso.fit, type='HC')
partial.se_2 <- sqrt(diag(HCV.coefs))[2]

Coefficient for D via partialling-out -0.08458483

In [27]:
table <- matrix(0,2,2)
table[1,1]<- partial.lasso.est_1
table[1,2]<-partial.se_1
table[2,1]<-partial.lasso.est_2
table[2,2]<-partial.se_2
colnames(table)<- c("Estimate","Std. Error")
rownames(table)<- c("partial reg via lasso(1)","partial reg via lasso(2)")
tab<- xtable(table, digits=c(3, 3, 4))
tab

Estimate    Std. Error
partial reg via lasso(1) -0.09065628 0.03223961
partial reg via lasso(2) -0.08458483 0.03248088